In [8]:
from BetaShF import ShF
from BetaShM import ShM 
import numpy as np 
import matplotlib.pyplot as plt
import time

In [9]:
def logSample(prefix, sample, fitness, force, moment):
    def appendToFile(name, text): 
        with open("./experiments/"+name, "a") as f:
            f.write(text + '\n')
    s = ""
    for x in sample: s += str(x) + " "
    appendToFile(str(prefix) + "_Population.txt", s)
    appendToFile(str(prefix) + "_Fitness.txt", str(fitness))
    appendToFile(str(prefix) + "_ShForces.txt", str(force))
    appendToFile(str(prefix) + "_ShMoments.txt", str(moment))

#### Objective Function

In [10]:
def objective_function(s, ShF, ShM, beta): #c is a constant that distributes the weight among the functions.
    print(f"Got {'-' * 15} {beta}")
    return beta*ShF(s) + (1-beta)*ShM(s)

In [11]:
# Bounds for each variable
nVar = 5
bounds = []
for i in range(1,nVar*3+1):
    if(i%3==0): bounds.append([0.005,0.04])
    else: bounds.append([-0.16, 0.16])
bounds = np.array(bounds)
print('bounds',bounds)

bounds [[-0.16   0.16 ]
 [-0.16   0.16 ]
 [ 0.005  0.04 ]
 [-0.16   0.16 ]
 [-0.16   0.16 ]
 [ 0.005  0.04 ]
 [-0.16   0.16 ]
 [-0.16   0.16 ]
 [ 0.005  0.04 ]
 [-0.16   0.16 ]
 [-0.16   0.16 ]
 [ 0.005  0.04 ]
 [-0.16   0.16 ]
 [-0.16   0.16 ]
 [ 0.005  0.04 ]]


#### GD

In [12]:
#################### Descenso de Gradiente con diferenciación finita
def Gradient_Descent(X0,f,bounds,MaxIter=1000,alpha=1e-3, beta = 0.5):
    eps = 1e-5
    def Gradiente(X,f, ShF, ShM, beta):
        n = len(X)
        G = np.zeros((n),float)
        incX = np.zeros((n),float)
        for i in range(n):
            incX[i] = eps
            G[i] = (f(X+incX, ShF, ShM, beta)-f(X, ShF, ShM, beta))/eps
            incX[i] = 0
        return G
    def getStepSize(alpha,beta,m,X,P,G,f):
        c0 = 1e-4
        c1 = 2
        c2 = 5
        c3 = 3
        eps = 1e-8
        while f(X+alpha*P, ShF, ShM, beta) > f(X, ShF, ShM, beta)+c0*np.dot(G,P):
            m = 0
            alpha = alpha/c1
            if alpha<=eps:
                break
        m += 1
        if m>=c2:
            m=0
            alpha = c3*alpha
        return alpha,m

    k=0
    X = X0
    G = Gradiente(X,f, ShF, ShM, beta)
    normaGradiente = np.linalg.norm(G)
    m = 0
    while(k<=MaxIter and normaGradiente>=eps):
        G = Gradiente(X0,f, ShF, ShM, beta)
        normaGradiente = np.linalg.norm(G)
        P = - G / normaGradiente
        alpha,m = getStepSize(alpha,beta,m,X,P,G,f)
        X = X + alpha*P
        X = np.clip(X,bounds[:,0],bounds[:,1])
        k = k+1
        if k%100 == 0:
            print("\t\t#",k,f(X, ShF, ShM, beta))
    return X
